## Step 1: Install and Import Required Libraries

In [ ]:

!pip install opencv-python face_recognition firebase-admin
import cv2
import face_recognition
import pickle
import os
import datetime
import firebase_admin
from firebase_admin import credentials, db, storage


## Step 2: Initialize Firebase

In [ ]:

def initialize_firebase():
    if not firebase_admin._apps:
        cred = credentials.Certificate("E:\cv\cv_lib\service2.json")
        firebase_admin.initialize_app(cred, {
            'databaseURL': "https://bigvision-22c68-default-rtdb.firebaseio.com/",
            'storageBucket': "bigvision-22c68.firebasestorage.app"
        })
    return db.reference('Students')


## Step 3: Setup Directories for Storing Images

In [ ]:

def setup_directories():
    if not os.path.exists('E:\cv\cv_lib\images'):
        os.makedirs('E:\cv\cv_lib\images')
    print("Directory 'E:\\cv\\cv_lib\\images' is ready")


## Step 4: Capture Face from Webcam

In [ ]:

x1, y1 = 363, 132
x2, y2 = 1002, 568
frame_width = x2 - x1
frame_height = y2 - y1

def capture_face():
    bg_img = cv2.imread("E:\cv\cv_lib\resources\register_bg.png")
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return None
    print("Please look at the camera... Press 'c' to capture or 'q' to quit")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break
        resized_frame = cv2.resize(frame, (frame_width, frame_height))
        output_img = bg_img.copy()
        output_img[y1:y2, x1:x2] = resized_frame
        face_locations = face_recognition.face_locations(frame)
        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.imshow('Face Registration System', output_img)
        key = cv2.waitKey(1)
        if key == ord('c'):
            if len(face_locations) == 0:
                print("No face detected. Try again.")
                continue
            if len(face_locations) > 1:
                print("Multiple faces detected. Only one person allowed.")
                continue
            print("Face captured successfully!")
            cap.release()
            cv2.destroyAllWindows()
            return frame
        elif key == ord('q'):
            print("Capture cancelled")
            cap.release()
            cv2.destroyAllWindows()
            return None


## Step 5: Enter Student Details

In [ ]:

def get_student_details():
    student_id = input("Enter student ID: ")
    name = input("Enter student name: ")
    major = input("Enter student major: ")
    year = int(input("Enter current year of study: "))
    return {
        "id": student_id,
        "name": name,
        "major": major,
        "year": year,
        "starting_year": datetime.datetime.now().year - year + 1,
        "total_attendance": 0,
        "standing": "G",
        "last_attendance_time": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }


## Step 6: Save Captured Face Image Locally

In [ ]:

def save_image(frame, student_id):
    if frame is None:
        return False
    filename = f"E:\cv\cv_lib\images\{student_id}.jpeg"
    cv2.imwrite(filename, frame)
    print(f"Image saved as {filename}")
    return True


## Step 7: Upload Image to Firebase Storage

In [ ]:

def upload_to_storage(student_id):
    local_path = f"E:\\cv\\cv_lib\\images\\{student_id}.jpeg"
    firebase_path = f"images/{student_id}.jpeg"
    if not os.path.exists(local_path):
        print(f"Error: File {local_path} does not exist")
        return False
    bucket = storage.bucket()
    blob = bucket.blob(firebase_path)
    blob.upload_from_filename(local_path)
    print(f"Uploaded {local_path} to Firebase")
    return True


## Step 8: Store Student Data in Firebase Database

In [ ]:

def store_in_database(ref, student_data):
    student_id = student_data["id"]
    data_to_store = student_data.copy()
    data_to_store.pop("id")
    ref.child(student_id).set(data_to_store)
    print(f"Student data for {student_id} stored in Firebase")
    return True


## Step 9: Update Face Encodings

In [ ]:

def update_encodings():
    print("Updating encodings...")
    folderPath = 'E:\cv\cv_lib\images'
    pathList = os.listdir(folderPath)
    imgList, studentIds = [], []
    for path in pathList:
        img = cv2.imread(os.path.join(folderPath, path))
        if img is None:
            continue
        imgList.append(img)
        studentIds.append(os.path.splitext(path)[0])
    encodeListKnown = []
    for img in imgList:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        try:
            encode = face_recognition.face_encodings(img)[0]
            encodeListKnown.append(encode)
        except IndexError:
            print("No face found in image. Skipping.")
            continue
    encodeListKnownWithIds = [encodeListKnown, studentIds]
    with open("E:\cv\cv_lib\EncodeFile.p", 'wb') as file:
        pickle.dump(encodeListKnownWithIds, file)
    print("Encodings saved successfully")
    return True


## Step 10: Register a New Student

In [ ]:

setup_directories()
ref = initialize_firebase()

while True:
    print("\nStarting registration...")
    frame = capture_face()
    if frame is None:
        if input("Try again? (y/n): ").lower() != 'y':
            break
        continue
    student_data = get_student_details()
    if not save_image(frame, student_data["id"]):
        continue
    if not upload_to_storage(student_data["id"]):
        continue
    if not store_in_database(ref, student_data):
        continue
    if not update_encodings():
        continue
    print(f"Student {student_data['name']} (ID: {student_data['id']}) registered successfully!")
    if input("Register another student? (y/n): ").lower() != 'y':
        break
print("Registration process completed.")
